In [6]:
import digitalocean
import paramiko
import digitalocean
import fabric
import bittensor
import os
import yaml
import json
from fabric import Connection

class Neuron:
    def __init__( self, name: str, sshkey:str, ip_address: str, wallet: 'bittensor.Wallet' ):
        self.name = name
        self.sshkey = sshkey
        self.wallet = wallet
        self.ip_address = ip_address
        self._connection = None
        self._debug = False

    def __str__(self) -> str:
        return "neuron({},{},{},{})".format( self.name, self.sshkey, self.ip_address, self.wallet)

    def __repr__(self) -> str:
        return self.__str__()

    def debug(self) -> 'Neuron':
        self._debug = True
        return self

    @staticmethod
    def load_from_digital_ocean( tag:str = None ) -> dict['Neuron']:
        manager = digitalocean.Manager( token = os.getenv( 'MARIUS_DOTOKEN' ))
        if tag is None:
            droplets = manager.get_all_droplets()
        else:
            droplets = manager.get_all_droplets( tag_name = [ tag ] )
        neurons = {}
        for drop in droplets:
            tag = drop.tags[0]
            name = "{}-{}".format( tag, drop.name )
            neurons[name] = Neuron ( 
                name, 
                sshkey = os.getenv( 'MARIUS_SSH_KEY' ),
                ip_address = drop.ip_address,
                wallet = bittensor.wallet( name = tag, hotkey = drop.name )
            )
        return neurons

    @staticmethod
    def load_all_from_config() -> dict['Neuron']:
        with open( 'config.yaml', "r") as config_file:
            config = yaml.safe_load(config_file)
        all = {}
        config = bittensor.Config.fromDict(config)
        for coldkey in config:
            for hotkey in config[coldkey]:
                name = "{}-{}".format( coldkey, hotkey )
                all[name] = Neuron(
                    name = name,
                    sshkey = config[coldkey][hotkey]['sshkey'],
                    ip_address = config[coldkey][hotkey]['ip_address'],
                    wallet = bittensor.wallet( name = coldkey, hotkey=hotkey)
                )
        return all

    @staticmethod
    def load_coldkey_from_config(coldkey:str) -> dict['Neuron']:
        with open( 'config.yaml', "r") as config_file:
            config = yaml.safe_load(config_file)
        all_coldkey = {}
        config = bittensor.Config.fromDict(config)
        for hotkey in config[coldkey]:
            name = "{}-{}".format( coldkey, hotkey )
            all_coldkey[name] = Neuron(
                name = name,
                sshkey = config[coldkey][hotkey]['sshkey'],
                ip_address = config[coldkey][hotkey]['ip_address'],
                wallet = bittensor.wallet( name = coldkey, hotkey=hotkey)
            )
        return all_coldkey

    @property
    def connection(self) -> Connection:
        if self._connection is None:
            key = paramiko.RSAKey.from_private_key_file( os.path.expanduser( self.sshkey ) )
            self._connection = Connection( self.ip_address, user='root', connect_kwargs={ "pkey" : key })
            return self._connection
        else:
            return self._connection

    @property
    def can_connect( self ) -> bool:
        try:
            result = self.connection.run('')
            return True
        except:
            return False
    
    def _check( self ):
        if not self.can_connect:
            raise Exception( "Can't connect to machine" )
    
    def run( self, cmd:str ):
        print ( "{}| Running: {}".format(self.name, cmd) )
        result = self.connection.run( cmd, hide = not self._debug)
        return result

    @property
    def is_installed( self ) -> bool:
        result = self.run( 'python3 -c "import bittensor"' )
        return result.ok

    @property
    def neuron(self) -> 'bittensor.Neuron':
        return self.get_neuron()

    @property
    def is_registered( self ) -> bool:
        neuron = self.get_neuron()
        return neuron.is_null

    @property
    def hotkey( self ) -> str:
        result = self.run( "cat ~/.bittensor/wallets/default/hotkeys/default" )
        if result.failed: return None
        else: return json.loads(result.stdout)['ss58Address']

    @property
    def coldkey( self ) -> str:
        result = self.run( "cat ~/.bittensor/wallets/default/coldkeypub.txt" )
        if result.failed: return None
        else: return json.loads(result.stdout)['ss58Address']

    @property
    def branch(self) -> str:
        result = self.run( 'cd ~/.bittensor/bittensor ; git branch --show-current' )
        if result.failed: return None
        else: return result.stdout

    @property
    def is_running( self ) -> bool:
        result = self.run( 'pm2 pid script')
        if len(result.stdout) > 1: return True
        else: return False  
    
    def install(self):
        self.run("sudo apt-get update && sudo apt-get install --no-install-recommends --no-install-suggests -y apt-utils curl git cmake build-essential gnupg lsb-release ca-certificates software-properties-common apt-transport-https")
        self.run("sudo apt-get install --no-install-recommends --no-install-suggests -y python3")
        self.run("sudo apt-get install --no-install-recommends --no-install-suggests -y python3-pip python3-dev python3-venv")
        self.run("ulimit -n 50000 && sudo fallocate -l 20G /swapfile && sudo chmod 600 /swapfile && sudo mkswap /swapfile && sudo swapon /swapfile && sudo cp /etc/fstab /etc/fstab.bak")
        self.run("sudo apt install npm -y")
        self.run("sudo npm install pm2@latest -g")
        self.run('mkdir -p ~/.bittensor/bittensor/')
        self.run('rm -rf ~/.bittensor/bittensor')
        self.run('git clone --recurse-submodules https://github.com/opentensor/bittensor.git ~/.bittensor/bittensor"')
        self.run('cd ~/.bittensor/bittensor ; pip3 install -e .')
        self.run("sudo apt-get update && sudo apt install docker.io -y && rm /usr/bin/docker-compose || true && curl -L https://github.com/docker/compose/releases/download/1.29.2/docker-compose-Linux-x86_64 -o /usr/local/bin/docker-compose && sudo chmod +x /usr/local/bin/docker-compose && sudo ln -s /usr/local/bin/docker-compose /usr/bin/docker-compose  && rm -rf subtensor || true && git clone https://github.com/opentensor/subtensor.git && cd subtensor && docker-compose up -d")
    
    def checkout_branch(self, branch):
        if "tags/" in branch:
            branch_str = "%s -b tag-%s" % (branch, branch.split("/")[1])
        else:
            branch_str = branch
        self.run( 'cd ~/.bittensor/bittensor ; git checkout %s' % branch_str )

    def pm2_show_script( self ):
        print( self.run( "pm2 show script" ) )

    def pm2_describe_script( self ):
        print( self.run( "pm2 describe script" ) )

    # write a function that returns the last n lines of the file /root/.pm2/logs/script-out.log on the machine
    def get_logs( self, lines:int ):
        result = self.run( "tail -n {} /root/.pm2/logs/script-out.log".format( lines ) )
        return result.stdout

    def clear_cache( self ):
        self.run( "rm -rf ~/.bittensor/miners && rm -rf /root/.pm2/logs/" )

    def reboot( self ):
        self.run( "sudo shutdown -r now -f")

    def get_neuron( self ) -> dict:
        sub = bittensor.subtensor(network='nakamoto')
        return sub.neuron_for_pubkey( ss58_hotkey = self.wallet.hotkey.ss58_address )

    # write a function that returns the CPU usage of the machine
    def get_cpu_usage( self ):
        result = self.run( "top -bn1 | grep load | awk '{printf \"CPU: %.2f\", $(NF-2)}'" )
        return result.stdout

    def load_wallet( self ):
        self.run( "mkdir -p ~/.bittensor/wallets/{}/hotkeys".format( self.wallet.name ) ) 
        self.run( "echo '{}' > ~/.bittensor/wallets/{}/hotkeys/{}".format( open(self.wallet.hotkey_file.path, 'r').read(), self.wallet.name, self.wallet.hotkey_str ) )
        self.run( "echo '{}' > ~/.bittensor/wallets/{}/coldkeypub.txt".format( open(self.wallet.coldkeypub_file.path, 'r').read(), self.wallet.name ) )

    def add_wallet( self, wallet: bittensor.Wallet ):
        self.run( "mkdir -p ~/.bittensor/wallets/{}/hotkeys".format( wallet.name ) ) 
        self.run( "echo '{}' > ~/.bittensor/wallets/{}/hotkeys/{}".format( open(wallet.hotkey_file.path, 'r').read(), wallet.name, wallet.hotkey_str ) )
        self.run( "echo '{}' > ~/.bittensor/wallets/{}/coldkeypub.txt".format( open(wallet.coldkeypub_file.path, 'r').read(), wallet.name ) )

    def register( 
            self, 
            wallet: 'bittensor.Wallet', 
            timeout:int = 10000 
        ):
        self.add_wallet( wallet )
        self.connection.run(
            "python3 ~/.bittensor/bittensor/bin/btcli register --wallet.name {} --wallet.hotkey {} --no_prompt".format( wallet.name, wallet.hotkey_str ), 
            warn=False, 
            hide=False, 
            disown = False, 
            timeout = 60
        )

    def stats(self) -> dict:
        neuron = self.get_neuron()
        self.stats = {
            'name': self.name,
            'ip_address': self.ip_address,
            'sshkey': self.sshkey,
            'wallet': self.wallet,
            'can_connect': self.can_connect,
            'branch': self.branch,
            'is_installed': self.is_installed,
            'is_running': self.is_running,
            'is_registered': neuron.is_null,
            'uid': neuron.uid,
            'stake': neuron.stake,
            'rank': neuron.rank,
            'trust': neuron.trust,
            'consensus': neuron.consensus,
            'incentive': neuron.incentive,
            'dividends': neuron.dividends,
            'emission': neuron.emission,
            'last_update': neuron.last_update,
            'active': neuron.active,
            'cpu_usage': self.get_cpu_usage(),
            'hotkey': self.hotkey,
            'coldkey': self.coldkey
        }
        return self.stats

    def rich_row( self, stats = None ) -> str:
        if stats == None:
            stats = self.stats()
        name_str = '[bold green]' + stats['name']
        ip_address_str = '[bold green]' + stats['ip_address']
        sshkey_str = '[bold green]' + stats['sshkey']
        hotkey_str =  '[bold green]' + stats['hotkey']
        coldkey_str =  '[bold green]' + stats['coldkey']
        branch_str = '[bold yellow] Yes' + stats['branch']
        can_connect_str = '[bold green] Yes' if stats['can_connect'] else '[bold red] No'
        is_installed_str = '[bold green] Yes' if stats['is_installed'] else '[bold red] No'
        is_running_str = '[bold green] Yes' if stats['is_running'] else '[bold red] No'
        is_registered_str = '[bold green] Yes' if stats['is_registered'] else '[bold red] No'
        metrics = [
            str( stats['uid'] ), 
            '{:.5f}'.format( stats['stake']),
            '{:.5f}'.format(  stats['rank']), 
            '{:.5f}'.format(  stats['trust']), 
            '{:.5f}'.format(  stats['consensus']), 
            '{:.5f}'.format(  stats['incentive']),
            '{:.5f}'.format(  stats['dividends']),
            '{:.5f}'.format(  stats['emission']),
            str( stats['last_update']),
            str( stats['active'] ), 
        ]
        return [ name_str, ip_address_str, sshkey_str, can_connect_str, branch_str, is_installed_str, is_running_str, is_registered_str] + metrics + [ coldkey_str, hotkey_str ]  

        

In [11]:
manager = digitalocean.Manager( token = os.getenv( 'MARIUS_DOTOKEN' ))
droplets = manager.get_all_droplets( tag_name = [ 'nukes' ] )
print (droplets)

[<Droplet: 288068284 N7>, <Droplet: 288068282 N5>, <Droplet: 288068286 N9>, <Droplet: 288068281 N4>, <Droplet: 288068283 N6>, <Droplet: 288068288 N3>, <Droplet: 288068279 N2>, <Droplet: 288068287 N0>, <Droplet: 288068285 N8>, <Droplet: 288068280 N1>]


In [5]:

n1.rich_row()


nukes-N0| Running: cd ~/.bittensor/bittensor ; git branch --show-current
nukes-N0| Running: python3 -c "import bittensor"
nukes-N0| Running: pm2 pid script
nukes-N0| Running: top -bn1 | grep load | awk '{printf "CPU: %.2f", $(NF-2)}'
nukes-N0| Running: cat ~/.bittensor/wallets/default/hotkeys/default
nukes-N0| Running: cat ~/.bittensor/wallets/default/coldkeypub.txt


['[bold green]nukes-N0',
 '[bold green]137.184.130.112',
 '[bold green]~/.ssh/marius',
 '[bold green] Yes',
 '[bold yellow] Yesmaster\n',
 '[bold green] Yes',
 '[bold green] Yes',
 '[bold red] No',
 '241',
 '160.54091',
 '0.00037',
 '0.12579',
 '0.02316',
 '0.00003',
 '0.00002',
 '0.00167',
 '1145592',
 '1',
 '[bold green]5GQU78jpAHypvyzHVxFJ9Uv8eYrommEtEKzwUXWUsnYFNpqn',
 '[bold green]5GmmudyYNpG566WphX8V7VDFuxqVD5hFSLqotqKHCorA3n2t']